# Using regular expressions to clean up the CAO points data

http://www.cao.ie/index.php?page=points&p=2021

In [1]:
# Importing the requests package so I can make HTTP requests
import requests as rq

#Import regualar expresions package
import re

#Import date and times. This allows to deal with date and time later.
import datetime as dt

print("done")

done


In [2]:
#fetches the cao points url
resp = rq.get('http://www2.cao.ie/points/l8.php')
#if the response is 200 everything is working ok so far
resp

<Response [200]>

## Saving the original data set

In [3]:
#getting the current date and time
now = dt.datetime.now()

#format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
#creating a file path for original data
path = 'data/cao2021_' + nowstr + '.html'

In [5]:
# fixing the encoding. The server on cao website uses the wrong encoding so changing it to cp1252.
original_encoding = resp.encoding

resp.encoding = 'cp1252'

In [6]:
#save the oringal html file to the path outlined above
with open(path, 'w') as f:
    f.write(resp.text)

In [7]:
#the r at the start of the string below means raw. It prevents the backslash from being interpreted.

re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')

 This is a regular expression here. It is looking to fully match a pattern. The first part [A-Z] is saying return any uppercase characters from a-z, the second part is a quantifier and it states that there are two digits(this is because the code names for the courses all start with two letter digts) The third part is a number 0-9 (all the college courses follow the two letter digits with numbers and this is folloowed by a quantifier of 3.

The line.decode part is to allow jupyter to read the lines as strings as opposed to bytes. Without this there would be an error.

In [8]:
path = 'data/cao2021_csv_' + nowstr + '.html'

no_lines = 0
#open the csv file for writing
with open(path, 'w') as f:
    #looping through the lines of the response content.
    for line in resp.iter_lines():
        #server on cao website says to decode with iso-8859-1. However, line that uses x96 isn't defined in iso-8859-1 so we are using the cp-1252 which includes x96.
        dline = line.decode('cp1252')
        #match the lines that represent courses.
        if re_course.fullmatch(dline):
            #add 1 to lines counter.
            no_lines = no_lines + 1
            
            #split the lines where there are two or more spaces.
            linesplit = re.split('  +', dline)
            #join the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

print(f"Total number of lines is {no_lines}.")

Total number of lines is 922.
